# River Tree

An example workflow for gathering NHD Plus data and generating a tree.

In [1]:
import os,sys
sys.path.append('/Users/ajc/Desktop/SimDataInputs/WorkSpace/ats-88/ats_meshing/workflow/')
import numpy as np
from matplotlib import pyplot as plt
import shapely
import logging
sys.path.append('/Users/ajc/anaconda3/envs/ats_meshing/')
import workflow
import workflow.source_list
import workflow.conf
import workflow.ui
import workflow.utils
import workflow.plot

workflow.conf.rcParams['data dir'] = os.path.join(os.getcwd(),'..','data')

workflow.ui.setup_logging(1,None)

In [2]:
# open a shapefile for use here
#use fiona to read shape file
manager_shp = workflow.source_list.FileManagerShape('../data/hydrologic_units/others/coweeta_basin/coweeta_basin.shp')
shp_profile,shp = manager_shp.get_shape()
shply = workflow.utils.shply(shp)

In [3]:
# find the rivers in this shape
#call to hilev.py
reaches, _ = workflow.get_rivers_by_bounds(workflow.source_list.hydrography_sources['NHD Plus'],
                                               shply.bounds, shp_profile['crs'], '0601', merge=False)

# filter the list to only those that intersect the shape
reaches = workflow.hydrography.filter_rivers_to_shape(shply, reaches, 10)

# make the global tree
rivers = workflow.hydrography.make_global_tree(reaches)

# check that only one tree was formed (this means al)
assert(len(rivers) is 1)
river = rivers[0]


2019-08-29 11:36:56,501 - root - INFO: 
2019-08-29 11:36:56,502 - root - INFO: Preprocessing Hydrography
2019-08-29 11:36:56,503 - root - INFO: ------------------------------
2019-08-29 11:36:56,504 - root - INFO: loading streams in bounds (273971.0911428096, 3878839.6361173145, 279140.9150949494, 3883953.7853134344)
2019-08-29 11:36:56,505 - root - INFO: Using Hydrography file "/Users/ajc/Desktop/SimDataInputs/WorkSpace/ats-88/ats_meshing/examples/../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb"
2019-08-29 11:37:02,962 - root - INFO:   ...filtering


In [4]:
%matplotlib
HSV_tuples=['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', 
            '#fabebe', '#008080', '#e6beff', '#9a6324', '#a9a9a9', '#800000', '#aaffc3', '#808000', '#ffd8b1', 
            '#000075', '#808080', '#000000', '#009090', '#090770']
workflow.plot.rivers([river,], shp_profile['crs'],color=HSV_tuples)
#print (river)
#for r in river:
#    print (r,len(river))
#    break

Using matplotlib backend: MacOSX


In [5]:
import colorsys
N = 22
HSV_tuples = [(x*1.0/N, x*.1/N, 1.0) for x in range(N)]
RGB_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples)

In [6]:
i = 0 
river_NHDPlusID = []
for r in river:
    #print (r.properties)
    river_NHDPlusID.append(r.properties['NHDPlusID'])
    #print(r.properties['LengthKM'], r.properties['Shape_Length'])
    #print (river['OderedDict']['Shape_Length'])
    #break
    
#Shape_Length: Feature length in decimal degrees 
#LengthKM: length in km
print (river_NHDPlusID)

[25000400040729.0, 25000400108020.0, 25000400040724.0, 25000400007107.0, 25000400007106.0, 25000400175089.0, 25000400074075.0, 25000400107798.0, 25000400108019.0, 25000400175304.0, 25000400040725.0, 25000400007320.0, 25000400074306.0, 25000400007109.0, 25000400074076.0, 25000400007108.0, 25000400040325.0, 25000400141373.0, 25000400175091.0, 25000400175090.0, 25000400208871.0, 25000400242778.0]


In [7]:
import fiona
fiona.listlayers('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb')

['ExternalCrosswalk',
 'FeatureToMetadata',
 'HUMod',
 'NHDFCode',
 'NHDFeatureToMetadata',
 'NHDMetadata',
 'NHDPlusDivFracMP',
 'NHDPlusEROMMA',
 'NHDPlusEROMQAMA',
 'NHDPlusEROMQARPT',
 'NHDPlusFlow',
 'NHDPlusFlowlineVAA',
 'NHDPlusIncrLat',
 'NHDPlusIncrPrecipMA',
 'NHDPlusIncrPrecipMM01',
 'NHDPlusIncrPrecipMM02',
 'NHDPlusIncrPrecipMM03',
 'NHDPlusIncrPrecipMM04',
 'NHDPlusIncrPrecipMM05',
 'NHDPlusIncrPrecipMM06',
 'NHDPlusIncrPrecipMM07',
 'NHDPlusIncrPrecipMM08',
 'NHDPlusIncrPrecipMM09',
 'NHDPlusIncrPrecipMM10',
 'NHDPlusIncrPrecipMM11',
 'NHDPlusIncrPrecipMM12',
 'NHDPlusIncrROMA',
 'NHDPlusIncrTempMA',
 'NHDPlusIncrTempMM01',
 'NHDPlusIncrTempMM02',
 'NHDPlusIncrTempMM03',
 'NHDPlusIncrTempMM04',
 'NHDPlusIncrTempMM05',
 'NHDPlusIncrTempMM06',
 'NHDPlusIncrTempMM07',
 'NHDPlusIncrTempMM08',
 'NHDPlusIncrTempMM09',
 'NHDPlusIncrTempMM10',
 'NHDPlusIncrTempMM11',
 'NHDPlusIncrTempMM12',
 'NHDPlusMegaDiv',
 'NHDPlusNHDPlusIDGridCode',
 'NHDProcessingParameters',
 'NHDReachCo

In [8]:
FlowlineVAA = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDPlusFlowlineVAA')
Flowline = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDFlowline')
Flow = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDPlusFlow')
Line = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDLine')
NHDArea = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDArea')

#FlowlineVAA[2] 
#Line[1]
#NHDArea[1]['properties']['AreaSqKm']
#print ('---------------')
#Waterbody[1]
#XX[1]
#FlowlineVAA[10]
#Flow[1]

#print (len(Flowline), len(FlowlineVAA), len(NHDArea),len(Waterbody), len(Flow))

In [9]:
NHDPLusEROMMA = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDPlusEROMMA')
#NHDPLusEROMMA[1]

In [10]:
import math

f_vector = lambda p,q: [q[0] - p[0], q[1] - p[1],  q[2] - p[2]]
#f_distance = lambda p,q: np.sqrt ((p[0] - q[0])**2 + (p[1] - q[1])**2 (p[2] - q[2])**2)
f_normal = lambda v,d: [v[0]/d, v[1]/d, v[2]/d]

def get_normal(coords):
    pos_v = f_vector(coords[0], coords[1])
    distance = math.sqrt(pos_v[0]**2 + pos_v[1]**2 + pos_v[2]**2)
    normal = f_normal(pos_v, distance)
    normal = [round(n,6) for n in normal]
    #print (normal)
    return normal



In [11]:
km_to_m = 1000.
def River_stream_length():
    #d = np.zeros((len(river),2))
    d2 = []
    for r in river:
        d1 = [('NHDPlusID',r.properties['NHDPlusID']), 
              ('LengthKM',r.properties['LengthKM']*km_to_m)]
        d2.append(d1)
    
    d3 = dict()
    d3['stream_lengthKM'] = d2
    return d3

In [12]:
DataLength = River_stream_length()
#print (len(DataLength))

In [20]:
#QAMA : Flow from runoff, MA: Mean Annual
#VANA : Velocity for QAMA
#TotDASqKm: TotalDrainageAreaSqKm [may not be what we want??]

def rating_curve(q_cfs,v_fs):
    b, f, m = 0.26, 0.40, 0.34
    w = x1 * q_cfs**b
    d = x2 * q_cfs**f
    v = x3 * q_cfs**m

In [63]:
DataFlow = dict()
discharge = []
velocity = []

Normal = []
segment_depth = []
segment_length = []
segment_width = []

km_to_m = 1000.
cfs_to_ms = 0.028316847 # QAMA is in cfs, convert cfs to m/s
fs_to_ms = 0.3048

# get discharge, velocity, depth, and orientation

def River_Characteristics():
    d4 = []
    for r in river:
        for i in range(len(NHDPLusEROMMA)):
            if NHDPLusEROMMA[i+1]['properties']['NHDPlusID'] == r.properties['NHDPlusID']:
                
                id = r.properties['NHDPlusID']
                
                value_q = NHDPLusEROMMA[i+1]['properties']['QAMA']*cfs_to_ms
                discharge.append([('NHDPlusID',id), ('Discharge [m^3/s]',value_q)])
                
                value_v = NHDPLusEROMMA[i+1]['properties']['VAMA']*fs_to_ms
                velocity.append([('NHDPlusID',id), ('Velocity [m/s]',value_v)])
                
                length = r.properties['LengthKM']*km_to_m
                
                #c = rating_curve(value_q,value_v)
                #break
                area = value_q/value_v#FlowlineVAA[i+1]['properties']['TotDASqKm']
                                
                
                width = 1.0 #area/length*km_to_m
                height = area * fs_to_ms #value_q/(value_v * width) 
                
                segment_length.append([('NHDPlusID',id), ('segment length [m]', length)])
                segment_depth.append([('NHDPlusID',id), ('segment depth [m]', height)])
                segment_width.append([('NHDPlusID',id), ('segment width [m]', width)])
                
                # orientation
                #if Flowline[i+1]['properties']['NHDPlusID'] == r.properties['NHDPlusID']:
                p = Flowline[i+1]['geometry']['coordinates'][0][0]
                q = Flowline[i+1]['geometry']['coordinates'][0][-1] #note coord are in reverse order in NHD 
                print (r.properties['NHDPlusID'],p,q)
                n = get_normal([p,q])
                Normal.append([n[0],n[1],n[2]])
                
                d4.append([('NHDPlusID',id), ('Discharge [m^3/s]',value_q),
                           ('Velocity [m/s]',value_v), ('segment length [m]', length),
                          ('segment depth [m]', height),('segment width [m]', width), 
                           ('orientation', [n[0],n[1],n[2]] )])
        d3 = dict()
        d3['stream_discharge'] = discharge
        d3['stream_velocity'] = velocity
        d3['stream_depth'] = segment_depth
        d3['stream_width'] = segment_width
        d3['stream_length'] = segment_length
        #d3['stream_area'] = segment_area
        d3['orientation'] = Normal 
    
    return d4


In [64]:
Data_NHD_ATS = River_Characteristics()

25000400040729.0 (-82.71182299559666, 35.67919499982713, 0.0) (-82.71173239455516, 35.67942909357674, 0.0)
25000400108020.0 (-83.70029158052068, 35.77817707779849, 0.0) (-83.69383398053071, 35.77354761113901, 0.0)
25000400040724.0 (-84.58805971247602, 36.20889867712992, 0.0) (-84.58759151247676, 36.207410277132226, 0.0)
25000400007107.0 (-83.3519605143947, 36.69010687638297, 0.0) (-83.35265578106032, 36.68963440971703, 0.0)
25000400007106.0 (-83.35726791438651, 36.65943480976392, 0.0) (-83.35946418104976, 36.65037667644464, 0.0)
25000400175089.0 (-82.80022124233449, 35.25069418174223, 0.0) (-82.79942460171071, 35.25056111611747, 0.0)
25000400074075.0 (-82.70622912477205, 35.89326530366151, 0.0) (-82.70714398831228, 35.89374045261911, 0.0)
25000400107798.0 (-82.30322338268923, 35.843553211030326, 0.0) (-82.30363531602194, 35.84449981102887, 0.0)
25000400108019.0 (-81.7963859168093, 36.640792476459524, 0.0) (-81.79807711680667, 36.64181087645795, 0.0)
25000400175304.0 (-82.59874508014718

In [65]:
Data_NHD_ATS[0]

[('NHDPlusID', 25000400040729.0),
 ('Discharge [m^3/s]', 0.9080831404970909),
 ('Velocity [m/s]', 0.339061961448),
 ('segment length [m]', 2825.0),
 ('segment depth [m]', 0.8163220080526847),
 ('segment width [m]', 1.0),
 ('orientation', [0.360939, 0.932589, 0.0])]

In [66]:
0.9080831404970909/0.339061961448

2.678221811196472

In [27]:

def Streams_connectivity():
    Node = []
    for r in river:
        for i in range(len(FlowlineVAA)):
            if FlowlineVAA[i+1]['properties']['NHDPlusID'] == r.properties['NHDPlusID']:
                id = r.properties['NHDPlusID']
                from_node = FlowlineVAA[i+1]['properties']['FromNode']
                to_node = FlowlineVAA[i+1]['properties']['ToNode']
                
                Node.append([('NHDPlusID',id), ('FromNode',from_node),('ToNode',to_node)])
    Nodes = dict()
    Nodes['connectivity'] = Node
    return Nodes



In [28]:
Connectivity = dict()
Connectivity_Orig = Streams_connectivity()

In [29]:
import copy
Connectivity =copy.deepcopy(Connectivity_Orig)
import networkx as nx

In [30]:
pairs = [(1, 2),(2, 3),(3, 4), (3, 5),(5, 9),(5, 7), (2, 8), (8,10), (8,11)]

graph_syn = nx.from_edgelist(pairs)
print (graph_syn.nodes)
#print (list(graph.neighbors(2)))
#print (len(graph))
#print(graph.number_of_edges())
#print(nx.node_boundary(graph, [2]))
#print(list(nx.edge_boundary(graph, [2])))
#list(nx.connected_components(graph))



[1, 2, 3, 4, 5, 9, 7, 8, 10, 11]


In [31]:
coweeta_pairs = []
#Coweeta_Graph=nx.Graph()
NHD_Id_Index = []
for i,n in enumerate(Connectivity_Orig['connectivity']):
    NHD_Id_Index.append([n[0][1],i+1])
    coweeta_pairs.append((int(n[2][1]),int(n[1][1])))
    #Coweeta_Graph.add_nodes_from([int(n[2][1]),int(n[1][1])])
Coweeta_Graph = nx.from_edgelist(coweeta_pairs)

coweeta_pairs_index = []
node_index_pair = []
#print (Coweeta_Graph.node)
for i,node in enumerate(Coweeta_Graph):
    node_index_pair.append((node,i+1))

#print (node_index_pair)
print (NHD_Id_Index)
print ('----------------------------')
def get_index_from_node(node):
    for n in node_index_pair:
        if node == n[0]:
            #print ('from node', node,n)
            break
    return n[1]

def get_node_from_index(index):
    for n in node_index_pair:
        if index == n[1]:
            print ('from index', index,n)
            break
    return n[0]

for n in Connectivity_Orig['connectivity']:
    x1 = get_index_from_node(int(n[2][1]))
    x2 = get_index_from_node(int(n[1][1]))
    coweeta_pairs_index.append((x1,x2))
    #print (n)
    
Coweeta_Graph1 = nx.from_edgelist(coweeta_pairs_index)

#for i,node in enumerate(Coweeta_Graph):
#    get_index_from_node(node)
#Coweeta_Graph1 = nx.from_edgelist(coweeta_pairs_index)
#print (Coweeta_Graph1.nodes)

[[25000400040729.0, 1], [25000400108020.0, 2], [25000400040724.0, 3], [25000400007107.0, 4], [25000400007106.0, 5], [25000400175089.0, 6], [25000400074075.0, 7], [25000400107798.0, 8], [25000400108019.0, 9], [25000400175304.0, 10], [25000400040725.0, 11], [25000400007320.0, 12], [25000400074306.0, 13], [25000400007109.0, 14], [25000400074076.0, 15], [25000400007108.0, 16], [25000400040325.0, 17], [25000400141373.0, 18], [25000400175091.0, 19], [25000400175090.0, 20], [25000400208871.0, 21], [25000400242778.0, 22]]
----------------------------


In [32]:
#25000400040725.0
#for i,node in enumerate(Coweeta_Graph1):
    
#get_node_from_index(3)

In [33]:
for node in Connectivity['connectivity']:
    print (node[0][1])
   
    if node[0][1] == 25000400040724.0:
        print ('----',node[0][1])
        break
    

25000400040729.0
25000400108020.0
25000400040724.0
---- 25000400040724.0


In [34]:
def treeDiagnostic(graph):
    for node in graph:
        edges,_ = node_get_upstream_edges(graph,[node])
        for edge in edges:
            print (edge)
            assert edge[0] < edge[1]

def numNodes(graph):
    return len(graph.nodes)


def isNeighborNode(graph,node):
    ngh_nodes = list(graph.neighbors(node))
    return  True if len(ngh_nodes) > 0 else False

def node_get_edges(graph, node):
    
    edges = list(nx.edge_boundary(graph, node))
    return edges

def node_get_upstream_edges(graph, node):
    edges = list(nx.edge_boundary(graph, node))
    #print (graph.nodes, node)
    parent = -100
    for i,edge in enumerate(edges):
        if edge[0] > edge[1]:
            parent = edges[i]
            edges.pop(i)
    return edges, parent

def isNeighborBoundaryNode(graph, edge):
    ngh = list(graph.neighbors(edge[1]))
    return False if len(ngh) > 1 else True

def isBoundaryNode(graph, node):
    assert (node != 1)
    #print (node)
    ngh = list(graph.neighbors(node))
    return False if len(ngh) > 1 else True

def get_node_index(graph, node):
    #node1 = int(node)
    print (node)
    x = np.where(np.array(graph.nodes) == node)[0][0] + 1
    return x

def index_based_graph(graph):
    Nodes_new_ids = np.linspace(1,len(graph),len(graph), dtype='i')
    Nodes = graph.nodes


    

In [ ]:
from lxml import etree
import lxml.builder
import lxml.etree as ET
Data = [10, 10, 10,10,10,10,10,10,10]
EnumNodes = []

def WriteXML3StreamSynthetic(data=Data):
    root = ET.Element("ParameterList", name="Main")
    segments = ET.SubElement(root,"ParameterList", name="segments")
    nam = ''
    node_id_root = 0
    for i,node in enumerate(graph_syn):
        print ('Main LOOP: ',node)
        depth = Data_NHD_ATS['stream_depth'][i]
        length = Data_NHD_ATS['stream_length'][i]
        if i ==0:
            node_id_root = node
                       
            edges = node_get_edges(graph_syn,[node])[0]
            print ('Edges', edges)
            nam = 'stream_' + str(edges[0]) + '_' + str(edges[1]) 
            #print ('NODE main: ', node, edges, nam)
            streams = ET.SubElement(segments,"ParameterList", name="%s"%nam)
            ET.SubElement(streams, "Parameter", name="segment length [m]", type="double", value="%s"%length)#['total length'][i])
            ET.SubElement(streams,"Parameter",name="number of cells", type="int", value="10")
            ET.SubElement(streams,"Parameter",name="orientation", type="Array(double)", value="{-1,0,0}")
            ET.SubElement(streams,"Parameter",name="cross sectional area [m^2]", type="double", value="%s"%depth)
            ET.SubElement(streams,"Parameter",name="first tip", type="Array(double)", value="{0.0,0.0,0.0}")
            ET.SubElement(streams,"Parameter",name="first tip type", type="string", value="boundary")
            ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="junction")
        else:
            nam = 'stream_' + str(node)
            edges, parent = node_get_upstream_edges(graph_syn,[node])
           
            edge_len = len(edges)
            
            if edge_len == 0: #boundary edge
                continue
                
            for j, child_node in enumerate(edges):
                
                if child_node[1] != node_id_root:
                    nam = 'stream_' + str(child_node[0]) + '_' + str(child_node[1])
                    streams = ET.SubElement(segments,"ParameterList", name="%s"%nam)
                    print ('ChildNode: ', child_node)
                    ET.SubElement(streams,"Parameter",name="first tip type", type="string", value="branch")
                    ET.SubElement(streams,"Parameter",name="first tip branch segment", type="string", value="stream_%s_%s"%(parent[1],parent[0]))
                    ET.SubElement(streams,"Parameter",name="first tip branch segment tip", type="string", value="last")
                    
                    if isBoundaryNode(graph_syn,child_node[1]):
                        ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="boundary")
                    else:
                        ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="junction")

                    
                    ET.SubElement(streams,"Parameter",name="orientation", type="Array(double)", value="{-1,0,0}")
                    ET.SubElement(streams, "Parameter", name="segment length [m]", type="double", value="%s"%length)
                    ET.SubElement(streams,"Parameter",name="number of cells", type="int", value="10") 
                    ET.SubElement(streams,"Parameter",name="cross sectional area [m^2]", type="double", value="%s"%depth)
                    
                    i = i + j
        
        if (i >= len(graph_syn)):
            break

    Data= etree.tostring(segments,encoding="unicode",pretty_print=True)
    outfile= open('river.xml','w')

    outfile.write(Data)



In [ ]:
#WriteXML3StreamSynthetic(Data)

In [56]:
from lxml import etree
import lxml.builder
import lxml.etree as ET
Data = np.ones(100)*19.
num_nodes = numNodes(Coweeta_Graph1)
fine_roots = []


def WriteXML3StreamNHD(data=Data):
    index = 0
    root = ET.Element("ParameterList", name="Main")
    ET.SubElement(root,"Parameter",name="infer cell centroids",type="bool", value="true")
    segments = ET.SubElement(root,"ParameterList", name="segments")
    nam = ''
    node_id_root = 0
    for i,node in enumerate(Coweeta_Graph1):
        if i ==1:
            break
        #print ('Main LOOP: ', node)
        
        
        length = Data_NHD_ATS[i][3][1]
        width = Data_NHD_ATS[i][5][1]
        orientation = Data_NHD_ATS[i][6][1]
        print (width)
        if i ==0:
            index = 0
            node_id_root = node
                       
            edges = node_get_edges(Coweeta_Graph1,[node])[0]
            
            nam = 'stream_' + str(edges[0]) + '_' + str(edges[1])
            print ('Edges ', edges,nam)
            streams = ET.SubElement(segments,"ParameterList", name="%s"%nam)
            ET.SubElement(streams, "Parameter", name="segment length [m]", type="double", value="%s"%length)
            #['total length'][i])
            ET.SubElement(streams,"Parameter",name="number of cells", type="int", value="10")
            
            n1,n2 = orientation[0],orientation[1]
            #Data_NHD_ATS['orientation'][index][0], Data_NHD_ATS['orientation'][index][1]
            ET.SubElement(streams,"Parameter",name="orientation", type="Array(double)", value="{%s,%s,0}"%(n1,n2))
            ET.SubElement(streams,"Parameter",name="cross sectional area [m^2]", type="double", value="%s"%width)
            ET.SubElement(streams,"Parameter",name="first tip", type="Array(double)", value="{0.0,0.0,0.0}")
            ET.SubElement(streams,"Parameter",name="first tip type", type="string", value="boundary")
            ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="junction")
        else:
            edges, parent = node_get_upstream_edges(Coweeta_Graph1,[node])
           
            edge_len = len(edges)
            
            if edge_len == 0: #boundary edge
                continue
                
            for j, child_node in enumerate(edges):
                
                if child_node[1] != node_id_root:
                    index = child_node[1] - 2 #index + 1
                    nam = 'stream_' + str(child_node[0]) + '_' + str(child_node[1])
                    streams = ET.SubElement(segments,"ParameterList", name="%s"%nam)
                    #print ('ChildNode: ', child_node, parent)
                    ET.SubElement(streams,"Parameter",name="first tip type", type="string", value="branch")
                    ET.SubElement(streams,"Parameter",name="first tip branch segment", type="string", value="stream_%s_%s"%(parent[1],parent[0]))
                    ET.SubElement(streams,"Parameter",name="first tip branch segment tip", type="string", value="last")
                    
                    if isBoundaryNode(Coweeta_Graph1,child_node[1]):
                        ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="boundary")

                    else:
                        ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="junction")
                    print (index,len(Data_NHD_ATS['orientation']))
                    n1,n2 = orientation[0],orientation[1]
                    #Data_NHD_ATS['orientation'][index][0], Data_NHD_ATS['orientation'][index][1]
                    ET.SubElement(streams,"Parameter",name="orientation", type="Array(double)", value="{%s,%s,0}"%(n1,n2))
                    ET.SubElement(streams, "Parameter", name="segment length [m]", type="double", value="%s"%length)
                    ET.SubElement(streams,"Parameter",name="number of cells", type="int", value="10") 
                    ET.SubElement(streams,"Parameter",name="cross sectional area [m^2]", type="double", value="%s"%width)
                    
                    fine_roots.append(nam)
                    
                    i = i + j
                
        if (i >= len(Coweeta_Graph1)):   
            break
    sets = ET.SubElement(root,"ParameterList",name="sets",type="bool", value="true")
    ET.SubElement(sets,"Parameter",name="fine_root",type="Array(string)", value="%s"%fine_roots)
    Data= etree.tostring(root,encoding="unicode",pretty_print=True)
    #outfile= open('coweeta_river-1.xml','w')
    outfile= open('one_segment.xml','w')

    outfile.write(Data)




In [57]:
WriteXML3StreamNHD()

1.0
Edges  (1, 2) stream_1_2


In [ ]:
import h5py
outfile='flux_pdepth.h5'
d=h5py.File(outfile,'w')

In [ ]:
from lxml import etree
import lxml.builder
import lxml.etree as ET

def WriteXML(data=Data):
    root = ET.Element("ParameterList", name="Main")
    segments = ET.SubElement(root,"ParameterList", name="segments")
    nam = ''
    for i in range(22):
        if i ==0:
            nam = 'main_stream'
        else:
            nam = 'stream_' + str(i)
        streams = ET.SubElement(segments,"ParameterList", name="%s"%nam)
        ET.SubElement(streams, "Parameter", name="total length", type="double", value="%s"%data['total length'][i])
        ET.SubElement(streams,"Parameter",name="number of cells", type="int", value="10")
        ET.SubElement(streams,"Parameter",name="cross sectional area [m^2]", type="double", value="6.87")
        ET.SubElement(streams,"Parameter",name="first tip", type="Array(double)", value="{0.0,0.0,0.0}")
        ET.SubElement(streams,"Parameter",name="first tip type", type="string", value="boundary")
        ET.SubElement(streams,"Parameter",name="last tip", type="Array(double)", value="{1000.0,0.0,0.0}")
        ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="boundary")

    Data= etree.tostring(segments,encoding="unicode",pretty_print=True)
    outfile= open('river.xml','w')

    outfile.write(Data)


WriteXML(Data)